In [2]:
import numpy as np
import pandas as pd

In [3]:
# 此方法產生的點擊資料不合理 相鄰重複
def data_generate1(size = 10000,leng = 11):
    data = np.array([chr(i) for i in range(65,65 + leng -1)])
    number = np.random.randint(10, size = (size,leng))
    count = 0
    path = np.array("←".join(data)).repeat(size)
    user = np.array("user#" + str(size)).repeat(size)
    for i in number[:,0]:
        path[count] = "←".join(data[number[count,1:(i+2)]])
        user[count] = "user#"+str(count+1)
        count += 1
    return path,user

In [4]:
# 相鄰不重複資料產生
# 隨機產生一個值為 0~9 相鄰不重複的陣列
def data_generate2(range_ = 9):
    import random
    arr = np.ones(1)
    long = random.randint(2, 10)
    for i in range(long):
        b = random.randint(0, range_)
        if (arr[len(arr)-1] == b):
            i -= 1
        else:
            arr = np.append(arr,b)
    return arr
def data_merge(size = 10000, leng = 11):
    import numpy as np
    import random
    data = np.array([chr(i) for i in range(65,65 + leng -1)])
    count = 0
    path = np.array("←".join(data)).repeat(size)
    user = np.array("user#" + str(size)).repeat(size)
    for i in range(size):
        temp = np.round(data_generate2()).astype(int)
        path[count] = np.array("←".join(data[temp[1:]]))
        user[count] = "user#"+str(count+1)
        if path[count] == "":
            path[count] = data[random.randint(0, 9)]      
        count += 1
    return path, user

In [5]:
path,user = data_merge(100000,11)
df = pd.DataFrame(path,columns = ["PATH"],index = user)

# 方法一
## 直接使用groupby進行計算

In [6]:
# 添加計數
df["quantity"] = 1
df.head()

,PATH,quantity
user#1,D←F←G←A,1
user#2,C←D,1
user#3,D←F,1
user#4,G←I,1
user#5,H←B←C←D←A←G←E,1


In [7]:
%timeit df.groupby("PATH").count().sort_values(by = "quantity",ascending = False)

127 ms ± 4.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
method1 = df.groupby("PATH").count().sort_values(by = "quantity",ascending = False)

In [9]:
method1[method1.index.str.len() == 9].head()

,quantity
PATH,
A←J←I←J←F,5
A←D←B←C←D,4
C←I←D←J←F,4
J←A←G←B←D,4
G←A←I←A←G,4


## 方法一缺點
1.資料量龐大的情況下,最為耗時(10萬筆數據 145 ms)<br/>

# 方法二
## 切分資料之後再進行groupby

In [10]:
df.head()

,PATH,quantity
user#1,D←F←G←A,1
user#2,C←D,1
user#3,D←F,1
user#4,G←I,1
user#5,H←B←C←D←A←G←E,1


In [11]:
df_2 = df["PATH"].str.split('←',expand = True)

In [12]:
df_2 = df_2.fillna("")
df_2.head()

,0,1,2,3,4,5,6,7,8,9
user#1,D,F,G,A,,,,,,
user#2,C,D,,,,,,,,
user#3,D,F,,,,,,,,
user#4,G,I,,,,,,,,
user#5,H,B,C,D,A,G,E,,,


In [13]:
df_2["quantity"] = 1
df_2.head()

,0,1,2,3,4,5,6,7,8,9,quantity
user#1,D,F,G,A,,,,,,,1
user#2,C,D,,,,,,,,,1
user#3,D,F,,,,,,,,,1
user#4,G,I,,,,,,,,,1
user#5,H,B,C,D,A,G,E,,,,1


In [14]:
# 篩選指定長度
df_len_5 = df_2[(df_2[5] != "") & (df_2[6] == "")]
df_len_5.head()

# 切分並還原字串
df_3 = pd.DataFrame(df_len_5[0] + "←" + df_len_5[1] + "←" + df_len_5[2] + "←" + df_len_5[3] + "←" + df_len_5[4] + "←" + df_len_5[5],columns = ["PATH"])
df_3["quantity"] = 1
df_3.head()

,PATH,quantity
user#29,J←D←F←G←A←E,1
user#43,F←J←G←E←C←E,1
user#45,C←G←F←D←B←G,1
user#64,H←B←D←A←B←I,1
user#66,G←B←F←B←I←G,1


In [15]:
# 另一種較迅速的方法
df_4 = df[(df['PATH'].str.len()>=5) & (df['PATH'].str.len() < 6)]

In [16]:
%timeit df_3.groupby("PATH").count()

14.7 ms ± 489 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 方法二缺點
1.操作繁瑣(使用df_4的方法較簡單)<br/>
2.當資料量龐大時仍然緩慢
### 優於方法一的部分在於事先切分資料,計算量較小